In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
def check_base_info(input_path, filename, output_path, minimal=False):
    df = pd.read_csv(input_path + filename + '.csv',  engine = 'python')
    col_names = df.columns.to_series()
    col_dtypes = df.dtypes
    col_na_count = df.isnull().sum()
    col_describe = df.describe().T
    col_unique = df.nunique()

    base_info = pd.concat([col_names, col_dtypes, col_na_count, col_unique], 
                          axis=1, keys=['col_names', 'col_dtypes', 
                                        'col_na_count', 'col_unique'], sort=False)
    base_info = pd.concat([base_info, col_describe], axis=1, sort=False)

    base_info.to_csv(output_path + filename + 'base_info.csv')
    
    df_report = ProfileReport(df, minimal=minimal)
    df_report.to_file(output_file = output_path + filename + '.html')

def check_missing(df):
    col_names = df.columns.to_series()
    col_dtypes = df.dtypes
    col_na_count = df.isnull().sum()
    col_describe = df.describe().T
    col_unique = df.nunique()

    base_info = pd.concat([col_names, col_dtypes, col_na_count, col_unique], 
                          axis=1, keys=['col_names', 'col_dtypes', 
                                        'col_na_count', 'col_unique'], sort=False)
    base_info = pd.concat([base_info, col_describe], axis=1, sort=False)
    
    return base_info

# 创建比赛ID，所有队伍交叉可能（不考虑区域），男队ID 为1000-1999, 数据中 ID数为1101-1467

In [6]:
input_path_mdata = f'./input/rawdata/MDataFiles_Stage1/'
input_path_mevents = f'./input/rawdata/'
output_path = f'./output/'

In [53]:
list_TeamID = [x for x in range(1101, 1468)]
list_Season = [x for x in range(2015, 2020)]

ID = []
Season = []
TeamID_a = []
TeamID_b = []

for item in  itertools.product(list_Season, list_TeamID, list_TeamID):
    if item[1] < item[2]:
        _ = str(item[0]) + '_' + str(item[1]) + '_' + str(item[2]) 
        ID.append(_)
        Season.append(item[0])
        TeamID_a.append(item[1])
        TeamID_b.append(item[2])

        
df_datamart = pd.concat([pd.Series(ID), pd.Series(Season), 
                pd.Series(TeamID_a), pd.Series(TeamID_b)], 
               axis=1, keys=['ID', 'Season', 'TeamID_a', 'TeamID_b' ])

In [57]:
check_missing(df_datamart)

,col_names,col_dtypes,col_na_count,col_unique,count,mean,std,min,25%,50%,75%,max
ID,ID,object,0,335805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Season,Season,int64,0,5,335805.0,2017.000000,1.414216,2015.0,2016.0,2017.0,2018.0,2019.0
TeamID_a,TeamID_a,int64,0,366,335805.0,1222.666667,86.384284,1101.0,1150.0,1208.0,1284.0,1466.0
TeamID_b,TeamID_b,int64,0,366,335805.0,1345.333333,86.384284,1102.0,1284.0,1360.0,1418.0,1467.0


In [54]:
print(df_datamart.head(100))

print(df_datamart.shape)

                ID  Season  TeamID_a  TeamID_b
0   2015_1101_1102    2015      1101      1102
1   2015_1101_1103    2015      1101      1103
2   2015_1101_1104    2015      1101      1104
3   2015_1101_1105    2015      1101      1105
4   2015_1101_1106    2015      1101      1106
5   2015_1101_1107    2015      1101      1107
6   2015_1101_1108    2015      1101      1108
7   2015_1101_1109    2015      1101      1109
8   2015_1101_1110    2015      1101      1110
9   2015_1101_1111    2015      1101      1111
10  2015_1101_1112    2015      1101      1112
11  2015_1101_1113    2015      1101      1113
12  2015_1101_1114    2015      1101      1114
13  2015_1101_1115    2015      1101      1115
14  2015_1101_1116    2015      1101      1116
15  2015_1101_1117    2015      1101      1117
16  2015_1101_1118    2015      1101      1118
17  2015_1101_1119    2015      1101      1119
18  2015_1101_1120    2015      1101      1120
19  2015_1101_1121    2015      1101      1121
20  2015_1101

# 加入target_p1，target_p2 此部分为对阵比赛结果，target_p1为分差，用score_diff表示，target_p2为胜负，用win表示
### [MRegularSeasonDetaileResults.csv] [MNCAATourneyDetailedResults.csv]两个文件中提取出胜负结果及分差，另加一列来区别常规赛和淘汰赛

In [7]:
df_regular_results = pd.read_csv(input_path_mdata + 'MRegularSeasonDetailedResults.csv', engine = 'python')
df_tourney_results = pd.read_csv(input_path_mdata + 'MNCAATourneyDetailedResults.csv', engine = 'python')

In [8]:
df_regular_results.head(5)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [9]:
df_tourney_results.head(5)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


### 说明：文件 [MRegularSeasonDetaileResults.csv] [MNCAATourneyDetailedResults.csv] 是文件 【MRegularSeasonCompactResults.csv】【MNCAATourneyCompactResults.csv】的扩展版本
### 前8列完全相同，后面追加的为整场比赛中的统计数据（此处要跟event区分开，event是发生每个活动都记录，每场比赛统计多次， 而detaile是每场比赛结束时，统计一次总数）
### WTO-turnovers committed？？ WBlk-blocks？？
### 每列说明： WFGM - 投篮投中次数（所有，包括2分，3分，不含罚篮） WFGA-投篮次数（比如5投3中 的5） WFGM3-3分球投中次数 WFGA3-3分球投篮次数 WFTM-罚球中次数 WFTA-罚球次数 
### WOR-进攻篮板 WDR-防守篮板 WAst-助攻 WTO-turnovers committed？？ WStl-抢断 WBlk-blocks？？ WPF-犯规

In [10]:
#Add a lable variable to the two datasets

df_regular_results['Label'] = 'Reg'
df_tourney_results['Label']= 'Tourney'

In [11]:
df_regular_results.shape

(87504, 34)

In [12]:
#Only use data after 2015
df_regular_results = df_regular_results[df_regular_results['Season'] >= 2015]

In [13]:
df_regular_results.shape

(26986, 34)

## Feature Engineering - Adding features for DataMart

### 1. Adding Score Difference

In [44]:
def score_diff(dfin):
    df = dfin[['Season','WTeamID','LTeamID','WScore','LScore','Label']]
        
    df['id_diff'] = df['WTeamID'] - df['LTeamID'] #Variable used to create ID matched with datamart
    
    '''Rule: if Win team has smaller ID number than Lost team - > Score_diff = Wscore - LScore
            else: Score_diff = LScore - Wscore since the team ID order will be exchanged'''
    
    df['Score_diff'] = df.apply(lambda x: x.WScore - x.LScore if x.id_diff <0 else x.LScore-x.WScore, axis = 1) 
    
    for i in ['Season','WTeamID','LTeamID']:
        df[i] = df[i].apply(lambda x: str(x))
    
    df['ID'] = df.apply(lambda x: x.Season+'_'+x.WTeamID+'_'+x.LTeamID if x.id_diff <0 else x.Season +'_'+ x.LTeamID+'_'+x.WTeamID,axis= 1)
    
    return df

In [45]:
#Apply score_diff to both regular season and tourney
df_list = [df_regular_results,df_tourney_results]

df_sdiff = pd.DataFrame()
for i in df_list:
    temp = score_diff(i)
    print(temp.shape)
    df_sdiff = df_sdiff.append(temp)

print(df_sdiff.shape)
print(df_sdiff.head())

C:\Users\wongy\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\wongy\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\wongy\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

(87504, 9)
(1115, 9)
(88619, 9)
  Season WTeamID LTeamID  WScore  LScore Label  id_diff  Score_diff  \
0   2003    1104    1328      68      62   Reg     -224           6   
1   2003    1272    1393      70      63   Reg     -121           7   
2   2003    1266    1437      73      61   Reg     -171          12   
3   2003    1296    1457      56      50   Reg     -161           6   
4   2003    1400    1208      77      71   Reg      192          -6   

               ID  
0  2003_1104_1328  
1  2003_1272_1393  
2  2003_1266_1437  
3  2003_1296_1457  
4  2003_1208_1400  


In [56]:
#Keep only ID and Score_diff and merged with DataMart

df_sdiff_formerge = df_sdiff[['ID','Score_diff','Label']]

df_datamart_add1 = pd.merge(left = df_datamart,right = df_sdiff_formerge, on = 'ID',how = 'left')

print(df_datamart_add1.shape)

print(df_datamart_add1.head())

(343776, 6)
               ID  Season  TeamID_a  TeamID_b  Score_diff Label
0  2015_1101_1102    2015      1101      1102         NaN   NaN
1  2015_1101_1103    2015      1101      1103         NaN   NaN
2  2015_1101_1104    2015      1101      1104         NaN   NaN
3  2015_1101_1105    2015      1101      1105         NaN   NaN
4  2015_1101_1106    2015      1101      1106         NaN   NaN


In [58]:
check_missing(df_datamart_add1)

,col_names,col_dtypes,col_na_count,col_unique,count,mean,std,min,25%,50%,75%,max
ID,ID,object,0,335805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Season,Season,int64,0,5,343776.0,2017.000108,1.414250,2015.0,2016.0,2017.0,2018.0,2019.0
TeamID_a,TeamID_a,int64,0,366,343776.0,1222.709927,86.379098,1101.0,1150.0,1208.0,1284.0,1466.0
TeamID_b,TeamID_b,int64,0,366,343776.0,1345.352904,86.354654,1102.0,1284.0,1360.0,1418.0,1467.0
Score_diff,Score_diff,float64,316455,132,27321.0,-0.804912,15.113810,-69.0,-11.0,-2.0,9.0,73.0
Label,Label,object,316455,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
